In [1]:
import sys, pygame
from pygame import Color, surfarray
import numpy as np
import random

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


### Description:
- states list:
  - the states list holds discrete boolean values, not RGB colors.
  - Bottom state starts at 0 for States obj, Top at -1. States are reversed when read to the screen.
- colors list:
  - the colors list holds rows of RGB tuples.
  - colors are translated from the states list using the color_dict.

In [89]:
cell_width = 90
cell_height = 90

cell_size = 10 # Number of pixels per cell

display_width = cell_width*cell_size
display_height = cell_height*cell_size

color_dict = {
        0: (255, 255, 255),
        1: (0, 0, 0)
        }

n_bits = 3

EVOLVE_COLOR = True

In [20]:
def init_states(itype='random', return_btm=False):
    states = [[0]*cell_width for i in range(cell_height)] # Zero State
    if itype == 'random':
        states[0] = [random.randrange(2) for i in range(cell_width)] # Random State
    if return_btm:
        return states[0]
    return states

In [95]:
def next_gen(left, middle, right, nth_rule):
    left, middle, right = int(bool(left)), int(bool(middle)), int(bool(right))
    value = int(str(left) + str(middle) + str(right), 2)
    return get_rule(nth_rule)[value]

def new_state(states, nth_rule=30):
    new = states[0].copy() # Get bottom row
    
    if nth_rule == 'random':
        nth_rule = random.randrange(2**(2**n_bits))
    
    new[1:-1] = [next_gen(new[i-1], new[i], new[i+1], nth_rule) for i in range(1, len(new)-1)] # New row, keep edge values
    states.insert(0, new) # Add new row to state array
    states.pop() # Shift states up 1 row
    
    return states # return state array

"""
    Returns RGB list from state list
    - states: states object
    - colors: None=All pixels updated, colors=use prev colors object and only update btm row
"""
def states_to_pix(states, colors=None):
    if colors == None:
        return [[color_dict[c] for c in s] for s in states] # All state colors will match current color dict mapping
    else:
        btm = states[0] # Bottom
        
        # Add new row to color list with current color & shift FIFO
        colors.insert(0, [color_dict[s] for s in btm])
        colors.pop()
        return colors # Use all colors from colors, but update bottom row with new colors

In [22]:
def get_rules(n_bits=3, nth_rule=None):
    n_states = 2**n_bits
    rules = np.zeros((2**n_states, n_states))
    for i in range(2**n_states):
        rule = format(i, '0'+str(n_states)+'b')
        rules[i,:] = np.array([int(b) for b in rule])
        
    if nth_rule != None:
        return rules[nth_rule, :]
    
    return rules

def get_rule(nth_rule, n_bits=3):
    n_states = 2**n_bits
    rule = format(nth_rule, '0'+str(n_states)+'b')
    return np.array([int(b) for b in rule])

In [100]:
def init_color_dict(color='random'):
    if color == 'random':
        color_dict[1] = (random.randrange(255), random.randrange(255), random.randrange(255))
    else:
        color_dict[1] = (0, 0, 0)

def update_color_dict(color='random', max_delta=10):
    if color == 'random':
        rgb = random.randrange(3) # Random Channel
        delta = random.randrange(-max_delta, max_delta) # Random change in color
        
        channels = list(color_dict[1])
        channel = channels[rgb]     
        channel += delta
        channel = channel if channel >= 0 else 0
        channel = channel if channel <= 255 else 255
        channels[rgb] = channel
        color_dict[1] = tuple(channels)

In [103]:
states = init_states() # Initialize States

init_color_dict() # Start with random color for on state
colors = states_to_pix(states) # Initialize colors

pygame.init()

screen = pygame.display.set_mode((display_width, display_height))
pygame.display.set_caption('1D Cellular Automata')

#create a surface with the size as the array
surface = pygame.Surface((cell_width, cell_height))
surface = pygame.transform.scale(surface, (display_width, display_height)) # Scale Size Up


run = True
cnt = 0
nth_rule = 0
while run:
    for event in pygame.event.get():
        if event.type == pygame.QUIT: 
            run = False
            #sys.exit()

    pygame.time.delay(5)
    
    # Reset bottom row if limited complexity
    if sum(states[0])/len(states[0]) < 0.05:
        states[0] = init_states(return_btm=True)
    
    if cnt % cell_height == cell_height-1:
        nth_rule += 1
        print('Rule:', nth_rule)
    
    # Scale Screen Size Down to edit pixel array at cell size=1 --------------------------------------------------
    surface = pygame.transform.scale(surface, (cell_width, cell_height))
    px = pygame.PixelArray(surface)
    
    # Translate States to Surface as RGB Colors
    states = new_state(states, nth_rule=nth_rule)
    colors = states_to_pix(states, colors=colors)
    for i, r in enumerate(colors):
        px[:,i] = colors[len(colors) - 1 - i]

    px.close()
    
    # Scale Screen Size Up to display pixel array at cell size=multiplier ----------------------------------------
    surface = pygame.transform.scale(surface, (display_width, display_height))  # Scale Size Up     
    screen.blit(surface, (0, 0)) # Update all pixels on screen object
    
    pygame.display.update()
    
    if EVOLVE_COLOR:
        update_color_dict(max_delta=40)
    cnt += 1
    
pygame.quit()

Rule: 1
Rule: 2
Rule: 3
Rule: 4
Rule: 5
Rule: 6
Rule: 7
Rule: 8
Rule: 9
Rule: 10
Rule: 11
Rule: 12
Rule: 13
Rule: 14
Rule: 15
Rule: 16
Rule: 17
Rule: 18
Rule: 19
Rule: 20
Rule: 21
Rule: 22
Rule: 23
Rule: 24
Rule: 25
Rule: 26
Rule: 27
Rule: 28
Rule: 29
Rule: 30
Rule: 31
Rule: 32
Rule: 33
Rule: 34
Rule: 35
Rule: 36
Rule: 37
Rule: 38
Rule: 39
Rule: 40
Rule: 41
Rule: 42
Rule: 43
Rule: 44
Rule: 45
Rule: 46
Rule: 47
Rule: 48
Rule: 49
Rule: 50
Rule: 51
Rule: 52
Rule: 53
Rule: 54
Rule: 55
Rule: 56
Rule: 57
Rule: 58
Rule: 59
Rule: 60
Rule: 61
Rule: 62
Rule: 63
Rule: 64
Rule: 65
Rule: 66
Rule: 67
Rule: 68
Rule: 69
Rule: 70
Rule: 71
Rule: 72
Rule: 73
Rule: 74
Rule: 75
Rule: 76
Rule: 77
Rule: 78
Rule: 79
Rule: 80
Rule: 81
Rule: 82
Rule: 83
Rule: 84
Rule: 85
Rule: 86
Rule: 87
Rule: 88
Rule: 89
Rule: 90
Rule: 91
Rule: 92
Rule: 93
Rule: 94
Rule: 95
Rule: 96
Rule: 97
Rule: 98
Rule: 99
Rule: 100
Rule: 101
Rule: 102
Rule: 103
Rule: 104
Rule: 105
Rule: 106
Rule: 107
Rule: 108
Rule: 109
Rule: 110
Rule: 11